Neste modelo foram implementados os leadtimes

In [236]:
# dados do problema

import pandas as pd

periodos = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Periods')
plantas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Plants', decimal=",")
linhas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Lines')
cds = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Dist. Centers', decimal=",")
clientes = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Customers', decimal=",")
produtos = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Products')
rotas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Routes', decimal=",")
demandas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Demands')
capabilidades = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Capabilities')
ktaxas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Rates')

#Conjuntos

In [237]:
P = plantas['name'].tolist() # set plants
W = cds['name'].tolist() # set warehouses
Lines = linhas['name'].tolist() # set Line names
plants_of_lines = linhas['plant'].tolist() # linhas de cada planta

In [238]:
p_l = dict(zip(Lines,plants_of_lines))

k = {}
for _, i in ktaxas.iterrows():
  # var = i.to_dict()
  line, period, size, rate = i['line'], i['period'], i['size'], i['rate']
  k[(size,line,period)] = rate

C = clientes['name'].tolist() # clientes
I = produtos['product'].tolist() # produtos
E = produtos['size'].tolist() # tamanhos

T = periodos['period'].tolist() #periodos


size_of_products = dict(zip(I,E)) #tamanho de cada produto
tamanhos = list(set(E))
products_per_size = {i:[] for i in tamanhos} # produtos por tamanho
for i in I:
  products_per_size[size_of_products[i]].append(i)


In [239]:
period_index = {}
print(T)
for i in range(len(T)):
  period_index[T[i]] = i

print(period_index)
periodos = list(period_index.keys())

[Timestamp('2025-07-16 00:00:00'), Timestamp('2025-07-17 00:00:00'), Timestamp('2025-07-18 00:00:00'), Timestamp('2025-07-19 00:00:00'), Timestamp('2025-07-20 00:00:00'), Timestamp('2025-07-21 00:00:00'), Timestamp('2025-07-22 00:00:00')]
{Timestamp('2025-07-16 00:00:00'): 0, Timestamp('2025-07-17 00:00:00'): 1, Timestamp('2025-07-18 00:00:00'): 2, Timestamp('2025-07-19 00:00:00'): 3, Timestamp('2025-07-20 00:00:00'): 4, Timestamp('2025-07-21 00:00:00'): 5, Timestamp('2025-07-22 00:00:00'): 6}


In [240]:
#relate products to sizes

def prod_size(prod=None, size=None):
  if prod != None:
    return size_of_products[prod]
  elif size != None:
    return products_per_size[size]

#Parametros

In [241]:
G_p = dict(zip(P, list(plantas['storage_capacity']))) # capacidade max das plantas
G_w = dict(zip(W, list(cds['storage_capacity']))) # capacidade max dos warehouses
taxas = k # taxas por linha linha plant0_1:{periodo_t:{size:12oz, rate:k}}
print(taxas)

{('24oz', 'Plant 0_1', Timestamp('2025-07-16 00:00:00')): 3117195, ('12oz', 'Plant 0_2', Timestamp('2025-07-16 00:00:00')): 2313865, ('9.1oz Sleek', 'Plant 0_3', Timestamp('2025-07-16 00:00:00')): 2785232, ('9.1oz Sleek', 'Plant 1_1', Timestamp('2025-07-16 00:00:00')): 2774624, ('24oz', 'Plant 1_2', Timestamp('2025-07-16 00:00:00')): 3118823, ('12oz', 'Plant 1_3', Timestamp('2025-07-16 00:00:00')): 1918712, ('24oz', 'Plant 1_4', Timestamp('2025-07-16 00:00:00')): 3223474, ('9.1oz Sleek', 'Plant 2_1', Timestamp('2025-07-16 00:00:00')): 2981648, ('9.1oz Sleek', 'Plant 2_2', Timestamp('2025-07-16 00:00:00')): 2980050, ('16oz', 'Plant 2_3', Timestamp('2025-07-16 00:00:00')): 2777332, ('24oz', 'Plant 0_1', Timestamp('2025-07-17 00:00:00')): 3328187, ('12oz', 'Plant 0_2', Timestamp('2025-07-17 00:00:00')): 2171705, ('9.1oz Sleek', 'Plant 0_3', Timestamp('2025-07-17 00:00:00')): 2999247, ('9.1oz Sleek', 'Plant 1_1', Timestamp('2025-07-17 00:00:00')): 2633891, ('24oz', 'Plant 1_2', Timestamp('

In [242]:
t_pc = {} # taxa de frete | indexado por t_pc[(planta_p, cliente_c)]
for _,i in rotas.iterrows():
  if i.origin[0] == "P" and i.destination[0] == "C":
    org, dest, cost = i.origin, i .destination, i.unit_freight_cost
    t_pc[(org, dest)] = cost

print(t_pc)

{('Plant 0', 'Customer 1'): 0.03829194, ('Plant 0', 'Customer 3'): 0.042583288, ('Plant 0', 'Customer 5'): 0.043655324, ('Plant 0', 'Customer 8'): 0.006998952, ('Plant 0', 'Customer 9'): 0.05071022, ('Plant 0', 'Customer 10'): 0.050408352, ('Plant 0', 'Customer 11'): 0.030268356, ('Plant 0', 'Customer 12'): 0.052566528, ('Plant 0', 'Customer 13'): 0.04162574, ('Plant 0', 'Customer 14'): 0.001392852, ('Plant 1', 'Customer 0'): 0.032241876, ('Plant 1', 'Customer 1'): 0.0317796, ('Plant 1', 'Customer 2'): 0.047520964, ('Plant 1', 'Customer 3'): 0.026388972, ('Plant 1', 'Customer 7'): 0.04661164, ('Plant 1', 'Customer 8'): 0.066524356, ('Plant 1', 'Customer 9'): 0.05089248399999999, ('Plant 1', 'Customer 13'): 0.08148574400000001, ('Plant 1', 'Customer 14'): 0.06163784, ('Plant 2', 'Customer 1'): 0.03208386, ('Plant 2', 'Customer 2'): 0.01285322, ('Plant 2', 'Customer 4'): 0.064638652, ('Plant 2', 'Customer 5'): 0.10816372, ('Plant 2', 'Customer 6'): 0.031985296, ('Plant 2', 'Customer 7'):

In [243]:
t_wc = {} # taxa de frete | indexado por t_wc[(warehouse_w, cliente_c)]
for _,i in rotas.iterrows():
  if i.origin[0] == "W" and i.destination[0] == "C":
    org, dest, cost = i.origin, i .destination, i.unit_freight_cost
    t_wc[(org, dest)] = cost
print(t_wc)

{('Warehouse 1', 'Customer 1'): 0.045099336, ('Warehouse 2', 'Customer 1'): 0.034815952, ('Warehouse 1', 'Customer 3'): 0.04784647600000001, ('Warehouse 2', 'Customer 3'): 0.041228428, ('Warehouse 1', 'Customer 5'): 0.039016792, ('Warehouse 2', 'Customer 5'): 0.047327892, ('Warehouse 1', 'Customer 8'): 0.004633931999999999, ('Warehouse 2', 'Customer 8'): 0.010784708, ('Warehouse 1', 'Customer 9'): 0.052783212, ('Warehouse 2', 'Customer 9'): 0.051566144, ('Warehouse 1', 'Customer 10'): 0.05621694, ('Warehouse 2', 'Customer 10'): 0.048485324, ('Warehouse 1', 'Customer 11'): 0.035204732, ('Warehouse 2', 'Customer 11'): 0.029348464, ('Warehouse 1', 'Customer 12'): 0.058453732, ('Warehouse 2', 'Customer 12'): 0.048734488, ('Warehouse 1', 'Customer 13'): 0.038198188, ('Warehouse 2', 'Customer 13'): 0.044987604, ('Warehouse 1', 'Customer 14'): 0.008038052, ('Warehouse 2', 'Customer 14'): 0.003280948, ('Warehouse 3', 'Customer 0'): 0.031390552, ('Warehouse 4', 'Customer 0'): 0.030433792, ('War

In [244]:
t_pp = {} # taxa de frete | indexado por t_pp[(planta_p, planta_q)]
for _,i in rotas.iterrows():
  if i.origin[0] == "P" and i.destination[0] == "P":
    org, dest, cost = i.origin, i .destination, i.unit_freight_cost
    t_pp[(org, dest)] = cost

print(t_pp)

{('Plant 0', 'Plant 1'): 0.063019492, ('Plant 0', 'Plant 2'): 0.06902108800000001, ('Plant 1', 'Plant 0'): 0.063019492, ('Plant 1', 'Plant 2'): 0.038471236, ('Plant 2', 'Plant 0'): 0.06902108800000001, ('Plant 2', 'Plant 1'): 0.038471236}


In [245]:
t_pw = {} # taxa de frete | indexado por t_pp[(planta_p, warehouse_w)]
for _,i in rotas.iterrows():
  if i.origin[0] == "P" and i.destination[0] == "W":
    org, dest, cost = i.origin, i .destination, i.unit_freight_cost
    t_pw[(org, dest)] = cost

print(t_pw)

{('Plant 0', 'Warehouse 1'): 0.006807488, ('Plant 0', 'Warehouse 2'): 0.003862948, ('Plant 0', 'Warehouse 3'): 0.059109712, ('Plant 0', 'Warehouse 4'): 0.065215416, ('Plant 0', 'Warehouse 5'): 0.063431484, ('Plant 0', 'Warehouse 6'): 0.07532846, ('Plant 0', 'Warehouse 7'): 0.07000131200000001, ('Plant 0', 'Warehouse 8'): 0.063981308, ('Plant 1', 'Warehouse 1'): 0.06931938400000001, ('Plant 1', 'Warehouse 2'): 0.06051522, ('Plant 1', 'Warehouse 3'): 0.004631631999999999, ('Plant 1', 'Warehouse 4'): 0.00565054, ('Plant 1', 'Warehouse 5'): 0.00277862, ('Plant 1', 'Warehouse 6'): 0.042381984, ('Plant 1', 'Warehouse 7'): 0.033906596, ('Plant 1', 'Warehouse 8'): 0.03715794, ('Plant 2', 'Warehouse 1'): 0.075719384, ('Plant 2', 'Warehouse 2'): 0.065282284, ('Plant 2', 'Warehouse 3'): 0.035746468, ('Plant 2', 'Warehouse 4'): 0.043993548, ('Plant 2', 'Warehouse 5'): 0.041249828, ('Plant 2', 'Warehouse 6'): 0.006386652, ('Plant 2', 'Warehouse 7'): 0.005578764, ('Plant 2', 'Warehouse 8'): 0.005070

In [246]:
# t_wp = {} # taxa de frete | indexado por t_wp[(warehouse_p, planta_q)]
# for _,i in rotas.iterrows():
#   if i.origin[0] == "W" and i.destination[0] == "P":
#     org, dest, cost = i.origin, i .destination, i.unit_freight_cost
#     t_wp[(org, dest)] = cost

# print(t_wp)

In [247]:
ld = {} # leadtimes
for _, r in rotas.iterrows():
  origem, destino, leadtime = r['origin'], r['destination'], r['leadtime']
  ld[(origem, destino)] = leadtime
print(ld)

{('Plant 0', 'Warehouse 1'): 1, ('Warehouse 1', 'Plant 0'): 1, ('Plant 0', 'Warehouse 2'): 0, ('Warehouse 2', 'Plant 0'): 0, ('Plant 0', 'Warehouse 3'): 5, ('Warehouse 3', 'Plant 0'): 5, ('Plant 0', 'Warehouse 4'): 5, ('Warehouse 4', 'Plant 0'): 5, ('Plant 0', 'Warehouse 5'): 5, ('Warehouse 5', 'Plant 0'): 5, ('Plant 0', 'Warehouse 6'): 6, ('Warehouse 6', 'Plant 0'): 6, ('Plant 0', 'Warehouse 7'): 6, ('Warehouse 7', 'Plant 0'): 6, ('Plant 0', 'Warehouse 8'): 5, ('Warehouse 8', 'Plant 0'): 5, ('Plant 0', 'Plant 1'): 5, ('Plant 0', 'Plant 2'): 6, ('Plant 0', 'Customer 1'): 3, ('Warehouse 1', 'Customer 1'): 4, ('Warehouse 2', 'Customer 1'): 3, ('Plant 0', 'Customer 3'): 4, ('Warehouse 1', 'Customer 3'): 4, ('Warehouse 2', 'Customer 3'): 3, ('Plant 0', 'Customer 5'): 4, ('Warehouse 1', 'Customer 5'): 3, ('Warehouse 2', 'Customer 5'): 4, ('Plant 0', 'Customer 8'): 1, ('Warehouse 1', 'Customer 8'): 0, ('Warehouse 2', 'Customer 8'): 1, ('Plant 0', 'Customer 9'): 4, ('Warehouse 1', 'Customer 9

In [248]:
total_demand = 0
D = {} # Demandas | indexado por D[(customer_c, product_i, period_t)]
for _,r in demandas.iterrows():
  client, prod, perio , demand = r['customer'], r['product'], r['period'], r['demand']
  D[(client, prod, perio)] = demand
  total_demand+=demand
print(total_demand)
print(D)


215000000
{('Customer 0', 'Product 12', Timestamp('2025-07-16 00:00:00')): 1000000, ('Customer 0', 'Product 21', Timestamp('2025-07-16 00:00:00')): 400000, ('Customer 0', 'Product 6', Timestamp('2025-07-16 00:00:00')): 600000, ('Customer 1', 'Product 4', Timestamp('2025-07-16 00:00:00')): 600000, ('Customer 1', 'Product 15', Timestamp('2025-07-16 00:00:00')): 400000, ('Customer 2', 'Product 16', Timestamp('2025-07-16 00:00:00')): 1400000, ('Customer 3', 'Product 1', Timestamp('2025-07-16 00:00:00')): 600000, ('Customer 3', 'Product 5', Timestamp('2025-07-16 00:00:00')): 1200000, ('Customer 3', 'Product 11', Timestamp('2025-07-16 00:00:00')): 1000000, ('Customer 4', 'Product 9', Timestamp('2025-07-16 00:00:00')): 1400000, ('Customer 4', 'Product 12', Timestamp('2025-07-16 00:00:00')): 1000000, ('Customer 4', 'Product 15', Timestamp('2025-07-16 00:00:00')): 1200000, ('Customer 5', 'Product 20', Timestamp('2025-07-16 00:00:00')): 1600000, ('Customer 6', 'Product 17', Timestamp('2025-07-16

In [249]:
holding_cost = 0.0001
M = 20 # penalizacao por falta a demanda
# truck_capacity = 250000
estoque_inicial_p = {i:0 for i in P}
estoque_inicial_w = {i:0 for i in W}

#Modelo

In [250]:
from pyomo.environ import *
model = ConcreteModel()

#inicializa os conjuntos
model.P = Set(initialize=P) # plantas
model.L = Set(initialize=Lines) # Linhas
model.W = Set(initialize=W) # armazens
model.C = Set(initialize=C) # clientes
model.I = Set(initialize=I) # produtos
model.T = Set(initialize=T) # periodos
model.E = Set(initialize=tamanhos) # tamanhos
model.Origins = model.W | model.P
model.Destinations = model.W | model.P | model.C

#inicializa os parâmetros
model.G_w = Param(model.W, initialize=G_w) # capacidade maxima de estoque no armzem w
model.G_p = Param(model.P, initialize=G_p) # capacidade maxima de estoque na planta p
model.h = Param(initialize=holding_cost) # custo de estoque por unidade por periodo
model.M = Param(initialize=M) # penalizador de demanda
# model.Q = Param(initialize=truck_capacity) # capacidade maxima por caminhao
model.k = Param(model.E,model.L,model.T, initialize=taxas, default= 0) # capacidade produtiva de cada linha
model.t_pp = Param(model.P, model.P, initialize=t_pp, default=1e20) # custos  de frete
model.t_pc = Param(model.P, model.C, initialize=t_pc, default=1e20) # custos de frete
model.t_wc = Param(model.W, model.C, initialize=t_wc, default=1e20) # custos de frete
model.t_pw = Param(model.P, model.W, initialize=t_pw, default=1e20) #custos de frete
# model.t_wp = Param(model.W, model.P, initialize=t_wp, default=1e20) #custos de frete
model.D = Param(model.C, model.I,model.T, initialize=D, default=0) # demandas
model.estoque_inicial_p = Param(model.P, initialize=estoque_inicial_p)
model.estoque_inicial_w = Param(model.W, initialize=estoque_inicial_w)
model.lead_time = Param(model.Origins, model.Destinations, initialize=ld, default = 20)

#incializar as variaveis

model.x = Var(model.I , model.L , model.T, domain=NonNegativeReals)
model.x_p = Var(model.I , model.P , model.T , domain=NonNegativeReals)
model.S_p = Var(model.I, model.P, model.T, domain=NonNegativeReals)
model.S_w = Var(model.I, model.W, model.T, domain=NonNegativeReals)
model.f_pc = Var(model.I, model.P, model.C, model.T, domain=NonNegativeReals)
model.f_wc = Var(model.I, model.W, model.C, model.T, domain=NonNegativeReals)
model.f_pp = Var(model.I, model.P, model.P, model.T ,domain=NonNegativeReals)
model.f_pw = Var(model.I, model.P, model.W, model.T  ,domain=NonNegativeReals)
model.f_wp = Var(model.I, model.W, model.P, model.T  ,domain=NonNegativeReals)
model.U = Var(model.C, model.I, model.T, domain=NonNegativeReals)

# NOVO: Variaveis inteiras para o número de caminhões
# model.y_pc = Var(model.I, model.P, model.C, model.T, domain=NonNegativeIntegers)
# model.y_wc = Var(model.I, model.W, model.C, model.T, domain=NonNegativeIntegers)
# model.y_pp = Var(model.I, model.P, model.P, model.T ,domain=NonNegativeIntegers)
# model.y_pw = Var(model.I, model.P, model.W, model.T  ,domain=NonNegativeIntegers)

In [251]:
# definir funcao objetivo

def total_cost(m):
      # custo de frete Planta-Cliente
      cost_f_pc = sum(m.t_pc[p,c]* m.f_pc[i,p,c,t]
                      for i in m.I for p in m.P for c in m.C for t in m.T)
      # custo de frete Armazem-Cliente
      cost_f_wc = sum(m.t_wc[w, c] * m.f_wc[i, w, c, t]
                    for i in m.I for w in m.W for c in m.C for t in m.T)

      # custo de frete Planta-Planta
      cost_f_pp = sum(m.t_pp[p1, p2] * m.f_pp[i, p1, p2, t]
                    for i in m.I for p1 in m.P for p2 in m.P if p1 != p2 for t in m.T) # Adicionado if p1 != p2 para evitar frete de planta para ela mesma

      # custo de frete Planta-Armazem e Armazem-Planta
      cost_f_pw_wp = sum(m.t_pw[p, w] * (m.f_pw[i, p, w, t] + m.f_wp[i,w,p,t])
                    for i in m.I for p in m.P for w in m.W for t in m.T)

      # custo de estoque no Armazem
      cost_h_w = sum(m.h * m.S_w[i,w,t]
                     for i in m.I for w in m.W for t in m.T)

      # custo de penalidade por demanda nao atendida
      cost_unmet_demand = m.M * sum(m.U[c,i,t]
                                    for c in m.C for i in m.I for t in m.T)

      return cost_f_pc + cost_f_wc + cost_f_pp + cost_f_pw_wp + cost_h_w + cost_unmet_demand


model.obj = Objective(rule=total_cost, sense=minimize)


In [252]:
# definir as restrições

# 1) restricao de producao
model.Producao = ConstraintList()
for t in model.T:
  for l in model.L:
      for e in model.E:
        model.Producao.add(expr = sum(model.x[i,l,t] for i in model.I if size_of_products.get(i) == e) <= model.k[e,l,t])

# Restrição para definir x_p como a soma da produção por planta
model.Definicao_x_p = ConstraintList()
for i in model.I:
  for p in model.P:
    for t in model.T:
      model.Definicao_x_p.add(expr = model.x_p[i, p, t] == sum(model.x[i, l, t] for l in model.L if p_l[l] == p))



# 2) restricao de balanco de estoque na planta
# Para o primeiro período (t=1), consideramos o estoque inicial como 0
model.Balanco_estoque_planta = ConstraintList()
for p in model.P:
  for i in model.I:
    # Assuming T starts from 1
    first_period = min(model.T)
    model.Balanco_estoque_planta.add(
        expr = model.estoque_inicial_p[p] + model.x_p[i, p, first_period] + sum(model.f_pp[i, q, p, T[period_index[first_period]- model.lead_time[q,p]]] for q in model.P if q != p and period_index[first_period] - model.lead_time[q,p]>=0)
        + sum(model.f_wp[i, w, p, T[period_index[first_period] - model.lead_time[w,p]]] for w in model.W if period_index[first_period] - model.lead_time[w,p]>=0)==
               model.S_p[i, p, first_period] + sum(model.f_pw[i, p, w, first_period] for w in model.W) + sum(model.f_pc[i, p, c, first_period] for c in model.C) + sum(model.f_pp[i, p, q, first_period] for q in model.P if q != p)
    )
# Para os períodos subsequentes (t > 1)
    for t in model.T:
      if t > min(model.T):
        previous_period = T[period_index[t] - 1]
        model.Balanco_estoque_planta.add(
            expr = model.S_p[i, p, previous_period] + model.x_p[i, p, t] + sum(model.f_pp[i, q, p, T[period_index[t]- model.lead_time[q,p]]] for q in model.P if q != p and period_index[t] - model.lead_time[q,p] >= 0)
            + sum(model.f_wp[i,w,p,T[period_index[t] - model.lead_time[w,p]]] for w in model.W if period_index[t] - model.lead_time[w,p] >=0) ==
                   model.S_p[i, p, t] + sum(model.f_pw[i, p, w, t] for w in model.W) + sum(model.f_pc[i, p, c, t] for c in model.C) + sum(model.f_pp[i, p, q, t] for q in model.P if q != p)
        )


# 3)restricao de balanco de estoque no armazem
model.Balanco_estoque_armazem = ConstraintList()
first_period = min(model.T)
for w in model.W:
  for i in model.I:
    model.Balanco_estoque_armazem.add(
        expr =  model.estoque_inicial_w[w]
        + sum(model.f_pw[i, p, w, T[period_index[first_period] - model.lead_time[p,w]]] for p in model.P if period_index[first_period] - model.lead_time[p,w] >= 0)
        == model.S_w[i, w, first_period] + sum(model.f_wp[i, w, p, first_period] for p in model.P) +sum(model.f_wc[i, w, c, first_period] for c in model.C)
    )
# Para os períodos subsequentes (t > 1)
    for t in model.T:
      if t > min(model.T):
        previous_period = T[period_index[t] - 1]
        model.Balanco_estoque_armazem.add(
            expr = model.S_w[i, w, previous_period] + sum(model.f_pw[i, p, w, T[period_index[t] - model.lead_time[p,w]]] for p in model.P if period_index[t]-model.lead_time[p,w] >= 0) ==
                   model.S_w[i, w, t] + sum(model.f_wc[i, w, c, t] for c in model.C) + sum(model.f_wp[i, w, p, t] for p in model.P)
        )

# 4) restricao de estoque maximo na planta
model.Estoque_maximo_p = ConstraintList()
for p in model.P:
  for t in model.T:
    model.Estoque_maximo_p.add(
        expr = sum(model.S_p[i, p, t] for i in model.I) <= model.G_p[p]
    )


# 5) restricao de estoque maximo no armazem
model.Estoque_maximo_w = ConstraintList()
for w in model.W:
  for t in model.T:
    model.Estoque_maximo_w.add(
        expr = sum(model.S_w[i, w, t] for i in model.I) <= model.G_w[w]
    )


# 6) restricao de demanda
model.Demanda = ConstraintList()
for c in model.C:
  for i in model.I:
    for t in model.T:
      model.Demanda.add(
          expr = sum(model.f_pc[i, p, c, T[period_index[t] - model.lead_time[p,c]]] for p in model.P if period_index[t] - model.lead_time[p,c] >= 0) + sum(model.f_wc[i, w, c, T[period_index[t]- model.lead_time[w,c]]] for w in model.W if period_index[t]- model.lead_time[w,c] >= 0) + model.U[c,i,t] >= model.D[c, i, t])


# # 7) restricao do estoque nao poder sair no mesmo dia que chegou
# model.Estoque_nao_sai_mesmo_dia = ConstraintList()
# for w in model.W:
#   for i in model.I:
#     for t in model.T:
#       if t > min(model.T):
#         previous_period = T[period_index[t] - 1]
#         model.Estoque_nao_sai_mesmo_dia.add(
#             expr = sum(model.f_wp[i, w, p, t] for p in model.P) + sum(model.f_wc[i,w,c,t] for c in model.C) <= model.S_w[i,w,previous_period]
#             )


In [253]:
import highspy

In [254]:
solver = SolverFactory('highs')

solver.solve(model)
# Obtém o valor do objetivo
objective_value = model.obj()
print(f"Valor do objetivo : {objective_value:.2f} ")

Valor do objetivo : 2717227191.75 


In [255]:
# Imprimir o valor das variáveis U[c,i,t]
unmet_demand_total = 0
unmet_demands = {}
u_df = {"Customer":[], "Product":[],"Period":[],"Unmet Demand":[]}
for c in model.C:
  for i in model.I:
    for t in model.T:
      U = model.U[c,i,t].value
      unmet_demand_total += U
      unmet_demands[(c,i,t)] = U
      # print(f"U[{c},{i},{t}] = {U}")
      if U >0:
        u_df["Customer"].append(c)
        u_df["Product"].append(i)
        u_df["Period"].append(t)
        u_df["Unmet Demand"].append(U)

print(f"Total unmet demand: {unmet_demand_total}")
pct_met_demand = ((total_demand - unmet_demand_total)/total_demand) * 100
print(f"Total cost of operation: {objective_value - unmet_demand_total*M}")
print(f"Percentage of demand met: {pct_met_demand:.2f}%")

Total unmet demand: 135764705.0
Total cost of operation: 1933091.7473797798
Percentage of demand met: 36.85%


In [256]:
print("Unmet Demands: ")
unmet_demands
u_df = pd.DataFrame(u_df)
u_df

Unmet Demands: 


,Customer,Product,Period,Unmet Demand
0,Customer 0,Product 6,2025-07-16,600000.0
1,Customer 0,Product 12,2025-07-16,1000000.0
2,Customer 0,Product 12,2025-07-17,1000000.0
3,Customer 0,Product 12,2025-07-21,1600000.0
4,Customer 0,Product 15,2025-07-18,800000.0
...,...,...,...,...
128,Customer 14,Product 12,2025-07-16,400000.0
129,Customer 14,Product 12,2025-07-18,1600000.0
130,Customer 14,Product 12,2025-07-21,1200000.0
131,Customer 14,Product 17,2025-07-17,1600000.0


In [257]:
# Create dictionaries to store variable values
x_data = {"Line": [], "Period": [],"Product": [], "Value": []}
x_p_data = { "Plant": [], "Period": [],"Product": [], "Value": []}
S_p_data = { "Plant": [], "Period": [],"Product": [], "Value": []}
S_w_data = { "Warehouse": [], "Period": [],"Product": [], "Value": []}
f_pc_data = {"Plant": [], "Period": [], "Customer": [], "Product": [], "Value": []}
f_wc_data = {"Warehouse": [], "Period": [], "Customer": [], "Product": [], "Value": []}
f_wp_data = {"Warehouse": [], "Period": [], "Plant": [], "Product": [], "Value": []}
f_pp_data = {"Plant_Origin": [], "Period": [], "Plant_Destination": [], "Product": [], "Value": []}
f_pw_data = {"Plant": [], "Period": [], "Warehouse": [], "Product": [], "Value": []}
u_df = {"Period": [], "Customer" : [],"Product" : [], "Unmet Demand" : []}

# Extract values from each variable
for l in model.L:
  for t in model.T:
    for i in model.I:
      prod = model.x[i, l, t].value
      if prod > 0:
        x_data["Line"].append(l)
        x_data["Period"].append(t)
        x_data["Product"].append(i)
        x_data["Value"].append(prod)

for p in model.P:
  for t in model.T:
    for i in model.I:
      prod = model.x_p[i, p, t].value
      if prod > 0:
        x_p_data["Plant"].append(p)
        x_p_data["Period"].append(t)
        x_p_data["Product"].append(i)
        x_p_data["Value"].append(prod)
      stock = model.S_p[i, p, t].value
      if stock > 0:
        S_p_data["Plant"].append(p)
        S_p_data["Period"].append(t)
        S_p_data["Product"].append(i)
        S_p_data["Value"].append(stock)



for w in model.W:
  for t in model.T:

    for i in model.I:
      stock = model.S_w[i, w, t].value
      if stock > 0:
        S_w_data["Warehouse"].append(w)
        S_w_data["Period"].append(t)
        S_w_data["Product"].append(i)
        S_w_data["Value"].append(stock)

    for c in model.C:
      for i in model.I:
        flow = model.f_wc[i, w, c, t].value
        if flow > 0:
          f_wc_data["Warehouse"].append(w)
          f_wc_data["Period"].append(t)
          f_wc_data["Customer"].append(c)
          f_wc_data["Product"].append(i)
          f_wc_data["Value"].append(flow)

    for p in model.P:
      for i in model.I:
        flow = model.f_wp[i, w, p, t].value
        if flow > 0:
          f_wp_data["Warehouse"].append(w)
          f_wp_data["Period"].append(t)
          f_wp_data["Plant"].append(p)
          f_wp_data["Product"].append(i)
          f_wp_data["Value"].append(flow)


for p in model.P:
  for t in model.T:
    for c in model.C:
      for i in model.I:
        flow = model.f_pc[i, p, c, t].value
        if flow > 0:
          f_pc_data["Plant"].append(p)
          f_pc_data["Period"].append(t)
          f_pc_data["Customer"].append(c)
          f_pc_data["Product"].append(i)
          f_pc_data["Value"].append(flow)

  for t in model.T:
    for p2 in model.P:
      if p != p2: # Assuming f_pp is only for flow between different plants
        for i in model.I:
          flow = model.f_pp[i, p, p2, t].value
          if flow > 0:
            f_pp_data["Plant_Origin"].append(p)
            f_pp_data["Period"].append(t)
            f_pp_data["Plant_Destination"].append(p2)
            f_pp_data["Product"].append(i)
            f_pp_data["Value"].append(flow)

  for t in model.T:
    for w in model.W:
      for i in model.I:
        flow = model.f_pw[i, p, w, t].value
        if flow > 0:
          f_pw_data["Plant"].append(p)
          f_pw_data["Period"].append(t)
          f_pw_data["Warehouse"].append(w)
          f_pw_data["Product"].append(i)
          f_pw_data["Value"].append(flow)


# print("Valores das variáveisc U[c,i,t]:")
for c in model.C:
  for t in model.T:
    for i in model.I:
      unmet = model.U[c,i,t].value
      if unmet > 0:
        u_df["Customer"].append(c)
        u_df["Period"].append(t)
        u_df["Product"].append(i)
        u_df["Unmet Demand"].append(unmet)


# Create DataFrames
x_df = pd.DataFrame(x_data)
x_p_df = pd.DataFrame(x_p_data)
S_p_df = pd.DataFrame(S_p_data)
S_w_df = pd.DataFrame(S_w_data)
f_pc_df = pd.DataFrame(f_pc_data)
f_wc_df = pd.DataFrame(f_wc_data)
f_wp_df = pd.DataFrame(f_wp_data)
f_pp_df = pd.DataFrame(f_pp_data)
f_pw_df = pd.DataFrame(f_pw_data)
u_df = pd.DataFrame(u_df)

# Write to Excel
with pd.ExcelWriter('./sample_data/decision_variables2.xlsx') as writer:
    x_df.to_excel(writer, sheet_name='x', index=False)
    x_p_df.to_excel(writer, sheet_name='x_p', index=False)
    S_p_df.to_excel(writer, sheet_name='S_p', index=False)
    S_w_df.to_excel(writer, sheet_name='S_w', index=False)
    f_pc_df.to_excel(writer, sheet_name='f_pc', index=False)
    f_wc_df.to_excel(writer, sheet_name='f_wc', index=False)
    f_wp_df.to_excel(writer, sheet_name='f_wp', index=False)
    f_pp_df.to_excel(writer, sheet_name='f_pp', index=False)
    f_pw_df.to_excel(writer, sheet_name='f_pw', index=False)
    u_df.to_excel(writer,sheet_name='U', index=False)